In [1]:
import pandas as pd
import pybedtools
from glob import glob

# Load peak files

In [7]:
import pybedtools 

peak_filepaths = sorted(glob('9.4_unioned_peaks/*.bed'))

peak_filepaths


file_id_to_bedtool = {}

for peak_filepath in peak_filepaths:    
    print(peak_filepath)
    file_id = peak_filepath.split('/')[-1].split('.tsv')[0]
    print('\t', file_id)
    file_id_to_bedtool[file_id] = pybedtools.BedTool(peak_filepath)

9.4_unioned_peaks/unioned_cleaned_peaks.bed
	 unioned_cleaned_peaks.bed


# Load regions in which to shuffle STAMP peaks

In [8]:
# Load regions
region_map_bedtool = pybedtools.BedTool('/projects/ps-yeolab3/ekofman/ReferenceData/peakcalling_regions/hg38/hg38_cellranger_region_map.bed')
len(region_map_bedtool)

620869

# Figure out which region each peak was in

In [9]:
def add_index(r):
    return '{}:{}-{}({})'.format(r.chrom, r.start, r.end, r.strand)

In [10]:
names = 'chrom   start   end     edit_fraction   strand  target_bases    edited_bases    num_edited_reads        total_reads_in_region   fraction_reads_edited   mean_depth      num_substrate_bases     subregion_coverage      subregion_conversions   region_coverage region_conversions      gene_coverage   gene_conversions        score'.split()
#names = ['{}_region'.format(l) for l in names]
names


['chrom',
 'start',
 'end',
 'edit_fraction',
 'strand',
 'target_bases',
 'edited_bases',
 'num_edited_reads',
 'total_reads_in_region',
 'fraction_reads_edited',
 'mean_depth',
 'num_substrate_bases',
 'subregion_coverage',
 'subregion_conversions',
 'region_coverage',
 'region_conversions',
 'gene_coverage',
 'gene_conversions',
 'score']

In [11]:
file_id_to_intersected = {}

for file_id, bedtool in file_id_to_bedtool.items():
    print(file_id)
    intersected = region_map_bedtool.intersect(bedtool, wa=True, wb=True).to_dataframe(names=['chrom_region', 'start_region', 'end_region', 'symbol_region', 'label_region', 'strand_region'] +names )

    intersected.index = intersected.apply(add_index, axis=1)
    intersected = intersected[(intersected.start >= intersected.start_region) & (intersected.end <= intersected.end_region)]
    intersected['peak_size'] = intersected['end'] - intersected['start']
    intersected = intersected[~intersected.index.duplicated(keep='first')]
    print('\t', len(intersected), len(bedtool))
    file_id_to_intersected[file_id] = intersected
    

unioned_cleaned_peaks.bed
	 11067 11123


# Permute the STAMP peaks within their respective region

In [12]:
import random


NUM_RAND_REGIONS = 100

def get_randomized_region(chrom, start, end, strand, peak_size):
    random_regions = []
    for n in range(NUM_RAND_REGIONS):
        random_start = random.randint(start, end-peak_size)
        random_end = random_start + peak_size
        
        random_region = '{}:{}-{}({})'.format(chrom, random_start, random_end, strand)
        random_regions.append(random_region)
    
    return random_regions
    

In [13]:
def get_random_regions(chrom, start, end, peak_start, peak_end, strand):
    peak_size = peak_end - peak_start
    return get_randomized_region(chrom, start, end, strand, peak_size)
    

In [14]:
file_id_to_intersected.keys()

dict_keys(['unioned_cleaned_peaks.bed'])

In [15]:
file_id_to_shuffle_dfs = {}


for file_id, intersected in file_id_to_intersected.items():
    print(file_id)
    
    all_regions = []
    all_indices = []

    for r in intersected.iterrows():
        index = r[0]
        r = r[1]
        chrom = r.loc['chrom_region']
        start = r.loc['start_region']
        end = r.loc['end_region']
        peak_start = r.loc['start']
        peak_end = r.loc['end']
        strand = r.loc['strand_region']

        try:
            n_regions = get_random_regions(chrom, start, end, peak_start, peak_end, strand)
            all_regions.append(n_regions)

            all_indices.append(index)
        except Exception as e:
            print(chrom, start, end, peak_start, peak_end, strand)
            print(e)
            
    shuffle_df = pd.DataFrame(all_regions, index=all_indices)
    shuffle_df = shuffle_df[~shuffle_df.index.duplicated(keep='first')]
    
    print('\t', len(shuffle_df), len(shuffle_df.columns))
    file_id_to_shuffle_dfs[file_id] = shuffle_df

unioned_cleaned_peaks.bed
	 11067 100


# From aggregate dfs containing shuffled information, extract each column as a random bed file and save

In [16]:
def expand_string_to_df(label):
    chrom = label.split(':')[0]
    start = label.split(':')[1].split('-')[0]
    end = label.split(':')[1].split('-')[1].split('(')[0]
    strand = label.split('(')[1].split(')')[0]
    return chrom, start, end, strand


file_id_to_list_of_random_instance_dfs = {}


for file_id, shuffle_df in file_id_to_shuffle_dfs.items():
    print(file_id)
    random_instance_dfs = []

    for random_instance_index in shuffle_df.columns:
        print('{}/{}'.format(random_instance_index, len(shuffle_df.columns)))
        random_instance = shuffle_df[[random_instance_index]]
        random_instance_df = pd.DataFrame(zip(*random_instance[random_instance_index].apply(expand_string_to_df))).T
        random_instance_df.columns = ['chrom', 'start', 'end', 'strand']
        random_instance_dfs.append(random_instance_df)

    file_id_to_list_of_random_instance_dfs[file_id] = random_instance_dfs

unioned_cleaned_peaks.bed
0/100
1/100
2/100
3/100
4/100
5/100
6/100
7/100
8/100
9/100
10/100
11/100
12/100
13/100
14/100
15/100
16/100
17/100
18/100
19/100
20/100
21/100
22/100
23/100
24/100
25/100
26/100
27/100
28/100
29/100
30/100
31/100
32/100
33/100
34/100
35/100
36/100
37/100
38/100
39/100
40/100
41/100
42/100
43/100
44/100
45/100
46/100
47/100
48/100
49/100
50/100
51/100
52/100
53/100
54/100
55/100
56/100
57/100
58/100
59/100
60/100
61/100
62/100
63/100
64/100
65/100
66/100
67/100
68/100
69/100
70/100
71/100
72/100
73/100
74/100
75/100
76/100
77/100
78/100
79/100
80/100
81/100
82/100
83/100
84/100
85/100
86/100
87/100
88/100
89/100
90/100
91/100
92/100
93/100
94/100
95/100
96/100
97/100
98/100
99/100


# Make directories into which to put shuffles

In [18]:
import os

for file_id, list_of_random_instance_dfs in file_id_to_list_of_random_instance_dfs.items():
    print(file_id)
    dir_name = '/projects/ps-yeolab3/ekofman/Hugo/Full_RBFOX2_and_SLBP/9.4_shuffled_union_peaks/{}'.format(file_id)

    try:
        os.mkdir(dir_name)
    except Exception as e:
        print(e)
        
    # Save beds
    
    for i, df in enumerate(list_of_random_instance_dfs):
        df.to_csv('{}/{}_shuffle{}.tsv'.format(dir_name, file_id, i), index=False, sep='\t')

unioned_cleaned_peaks.bed


In [19]:
sorted(glob('/projects/ps-yeolab3/ekofman/Hugo/Full_RBFOX2_and_SLBP/9.4_shuffled_union_peaks/*'))

['/projects/ps-yeolab3/ekofman/Hugo/Full_RBFOX2_and_SLBP/9.4_shuffled_union_peaks/unioned_cleaned_peaks.bed']

# Add sequences and motif presence

In [20]:
shuffled_peak_filepaths = [f for f in glob('/projects/ps-yeolab3/ekofman/Hugo/Full_RBFOX2_and_SLBP/9.4_shuffled_union_peaks/*/*') \
                           if 'with_sequence' not in f and 'motif_presence' not in f]
len(shuffled_peak_filepaths)

100

In [21]:
shuffled_peak_filepaths[0:3]

['/projects/ps-yeolab3/ekofman/Hugo/Full_RBFOX2_and_SLBP/9.4_shuffled_union_peaks/unioned_cleaned_peaks.bed/unioned_cleaned_peaks.bed_shuffle16.tsv',
 '/projects/ps-yeolab3/ekofman/Hugo/Full_RBFOX2_and_SLBP/9.4_shuffled_union_peaks/unioned_cleaned_peaks.bed/unioned_cleaned_peaks.bed_shuffle3.tsv',
 '/projects/ps-yeolab3/ekofman/Hugo/Full_RBFOX2_and_SLBP/9.4_shuffled_union_peaks/unioned_cleaned_peaks.bed/unioned_cleaned_peaks.bed_shuffle23.tsv']

In [22]:
from pyfaidx import Fasta
import re
fasta = '/projects/ps-yeolab3/ekofman/ReferenceData/hg38/cellranger-GRCh38-3.0.0/fasta/genome.fa'
FA = Fasta(fasta, rebuild=False)
import math

def get_sequence(r): 
    chrom = str(r.chrom)
    start = r.start
    end = r.end
    strand = r.strand
    
    sequence = FA[chrom][start:end].seq
    sequence = sequence.upper()
        
    return sequence

def get_extended_sequence(r): 
    chrom = str(r.chrom)
    
    midpoint = r.start + (int((r.end - r.start)/2))
    start = midpoint - 150
    end = midpoint + 150
    strand = r.strand
    
    sequence = FA[chrom][start:end].seq
    sequence = sequence.upper()
        
    return sequence


In [23]:

for peak_filepath in sorted(shuffled_peak_filepaths):
    print('Processing {}...'.format(peak_filepath))
    peak_df = pd.read_csv(peak_filepath, sep='\t')
    filename = peak_filepath.split('/')[-1]
    
    print('\tfilename is {}'.format(filename))
    
    folder = peak_filepath.split(filename)[0]
    print('\tfolder is {}'.format(folder))
    output_filepath = '{}/{}.with_sequence.bed'.format(folder, filename[0:-4])
    print(output_filepath)
    
    peak_df['sequence(+)'] = peak_df.apply(get_sequence, axis=1)
    peak_df['extended_sequence(+)'] = peak_df.apply(get_extended_sequence, axis=1)

    print('\tAssigned sequences')
    peak_df.to_csv(output_filepath, sep='\t', index=False)
    print('\tOutput file at {}'.format(output_filepath))

Processing /projects/ps-yeolab3/ekofman/Hugo/Full_RBFOX2_and_SLBP/9.4_shuffled_union_peaks/unioned_cleaned_peaks.bed/unioned_cleaned_peaks.bed_shuffle0.tsv...
	filename is unioned_cleaned_peaks.bed_shuffle0.tsv
	folder is /projects/ps-yeolab3/ekofman/Hugo/Full_RBFOX2_and_SLBP/9.4_shuffled_union_peaks/unioned_cleaned_peaks.bed/
/projects/ps-yeolab3/ekofman/Hugo/Full_RBFOX2_and_SLBP/9.4_shuffled_union_peaks/unioned_cleaned_peaks.bed//unioned_cleaned_peaks.bed_shuffle0.with_sequence.bed
	Assigned sequences
	Output file at /projects/ps-yeolab3/ekofman/Hugo/Full_RBFOX2_and_SLBP/9.4_shuffled_union_peaks/unioned_cleaned_peaks.bed//unioned_cleaned_peaks.bed_shuffle0.with_sequence.bed
Processing /projects/ps-yeolab3/ekofman/Hugo/Full_RBFOX2_and_SLBP/9.4_shuffled_union_peaks/unioned_cleaned_peaks.bed/unioned_cleaned_peaks.bed_shuffle1.tsv...
	filename is unioned_cleaned_peaks.bed_shuffle1.tsv
	folder is /projects/ps-yeolab3/ekofman/Hugo/Full_RBFOX2_and_SLBP/9.4_shuffled_union_peaks/unioned_clean

# Determine motif presence in shuffled sequence-assigned .bed files 

In [24]:


complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
def reverse_complement(seq):
    return "".join(complement.get(base, base) for base in reversed(seq))

def reverse(seq):
    return seq[::-1]

def motif_present(sequence, motif, strand):
    if strand == '+':
        return motif in sequence
    elif strand == '-':
        return reverse_complement(motif) in sequence

def motif_distance_from_center(sequence, motif, strand):
    #print(sequence)
    if strand == '+':
        search_term = motif
    elif strand == '-':
        search_term = reverse_complement(motif)
    
    if search_term in sequence:
        closest_position_in_first_half = 1000
        closest_position_in_second_half = 1000
        
        midpoint = math.ceil(len(sequence)/2)
        #print('\tmidpoint is {}'.format(midpoint))
        start_of_second_half = midpoint-len(motif)

        sequence_first_half = sequence[0:midpoint]
        sequence_second_half = sequence[start_of_second_half:]

        #print('\tSplit:', sequence_first_half, sequence_second_half)
        if search_term in sequence_first_half:
            flipped_sequence = reverse(sequence_first_half)
            flipped_search_term = reverse(search_term)
            
            #print('searching for {} in {}'.format(flipped_search_term, flipped_sequence))
            closest_position_in_first_half = - flipped_sequence.index(flipped_search_term) - math.floor(len(motif)/2)
            
        if search_term in sequence_second_half:
            closest_position_in_second_half = sequence_second_half.index(search_term) - math.floor(len(motif)/2)

        if min(abs(closest_position_in_second_half), abs(closest_position_in_first_half)) == abs(closest_position_in_second_half):
            return closest_position_in_second_half
        else:
            return closest_position_in_first_half
    else:
        return None
    
motifs = ['TGCATG']#, 'GAATG', 'GTTTG', 'GTGTG', 'GTATG', 'GCTTG', 'GCCTG']

def add_sequence_presences(r, window_size=70):
    extended_sequence = r['extended_sequence(+)']
    strand = r.strand

    distance = motif_distance_from_center(extended_sequence, 'TGCATG', strand)
    
    sequence = r['sequence(+)']

    presence_map = {motif: motif_present(sequence, motif, strand) for motif in motifs}
    #return presence_map.get(motifs[0]), presence_map.get(motifs[1]), presence_map.get(motifs[2]), presence_map.get(motifs[3]), presence_map.get(motifs[4]), presence_map.get(motifs[5]), presence_map.get(motifs[6]), distance
    return presence_map.get(motifs[0]), distance

def calculate_fractions(p_df):
    fractions_dict = {}
    counts_dict = {}
    
    for motif in motifs:# + ['any_motif']:
        motif_present_count = p_df[motif].sum()
        motif_present_fraction = motif_present_count/len(p_df)
        fractions_dict[motif] = motif_present_fraction
        counts_dict[motif] = motif_present_count
    
    return fractions_dict, counts_dict

### tests
print(motif_present('TGCATG', 'TGCATG', '+'))
print(motif_present('TGCATG', 'TGCATG', '-'))
print(motif_present('TGCATG', 'CATGCA', '-'))

print(reverse('GCATG'))

print(motif_distance_from_center('GCATGZZZZZZZZZZ', 'GCATG', '+'))
print(motif_distance_from_center('ZZZZZGCATGZZZZZ', 'GCATG', '+'))
print(motif_distance_from_center('ZZZZZZGCATGZZZZ', 'GCATG', '+'))
print(motif_distance_from_center('ZZZZZZZGCATGZZZ', 'GCATG', '+'))
print(motif_distance_from_center('ZZZZZZZZZZGCATG', 'GCATG', '+'))
print(motif_distance_from_center('GCATGZGCATGZZZZZ', 'BAB', '+'))
print(motif_distance_from_center('ZZZGCATGZZZGCATG', 'GCATG', '+'))
print(motif_distance_from_center('AAGCATGCATGCATGCATGCATGA', 'GCATG', '+'))



True
False
True
GTACG
-5
0
1
2
5
None
-2
1


In [25]:
shuffled_peak_with_sequence_filepaths = glob('/projects/ps-yeolab3/ekofman/Hugo/Full_RBFOX2_and_SLBP/9.4_shuffled_union_peaks/*/*with_sequence.bed')




In [26]:
len(shuffled_peak_with_sequence_filepaths)

100

In [27]:
all_fractions = pd.DataFrame()
all_counts = pd.DataFrame()

num_peak_per_sample_id = {}

for filepath in shuffled_peak_with_sequence_filepaths:
    # Extract sample ID from filepath
    sample_id = filepath.split('/')[-1].split('.with_sequence.bed')[0]
    
    if 'SLBP' in sample_id:
        continue
    else:
        folder = filepath.split(sample_id)[0]

        print('\t', folder, sample_id)

        output_filename = '{}/{}.motif_presence.bed'.format(folder, sample_id)
        print('\t\t', output_filename)

        print('\t...Reading...')
        p_df = pd.read_csv(filepath, sep='\t')
        num_peaks = len(p_df)
        num_peak_per_sample_id[sample_id] = num_peaks

        # Calculate sequence presence
        #p_df[motifs[0]],p_df[motifs[1]],p_df[motifs[2]],p_df[motifs[3]],p_df[motifs[4]],p_df[motifs[5]],p_df[motifs[6]],p_df['GCATG_dist_from_center'] = zip(*p_df.apply(add_sequence_presences, axis=1))
        p_df[motifs[0]],p_df['TGCATG_dist_from_center'] = zip(*p_df.apply(add_sequence_presences, axis=1))

        print('\tOutputting file {}'.format(output_filename))

        #p_df['any_motif'] = p_df[[motifs[0], motifs[1], motifs[2], motifs[3], motifs[4], motifs[5], motifs[6]]].any(axis=1)

        p_df.to_csv(output_filename, sep='\t', index=False, header=True)

        new_fractions_dict, new_counts_dict = calculate_fractions(p_df)

        new_fractions = pd.DataFrame.from_dict(new_fractions_dict, orient='index', columns=[sample_id])
        new_counts = pd.DataFrame.from_dict(new_counts_dict, orient='index', columns=[sample_id])    

        if all_fractions.empty:
            all_fractions = new_fractions
        else:
            all_fractions = all_fractions.join(new_fractions, how='inner')

        if all_counts.empty:
            all_counts = new_counts
        else:
            all_counts = all_counts.join(new_counts, how='inner')
        print('\t', len(all_fractions.columns))

all_fractions[sorted(all_fractions.columns)].to_csv('9.4_shuffled_union_peaks/FOX_cleaned_motif_presence_in_union_shuffles.tsv', sep='\t')



	 /projects/ps-yeolab3/ekofman/Hugo/Full_RBFOX2_and_SLBP/9.4_shuffled_union_peaks/unioned_cleaned_peaks.bed/ unioned_cleaned_peaks.bed_shuffle68
		 /projects/ps-yeolab3/ekofman/Hugo/Full_RBFOX2_and_SLBP/9.4_shuffled_union_peaks/unioned_cleaned_peaks.bed//unioned_cleaned_peaks.bed_shuffle68.motif_presence.bed
	...Reading...
	Outputting file /projects/ps-yeolab3/ekofman/Hugo/Full_RBFOX2_and_SLBP/9.4_shuffled_union_peaks/unioned_cleaned_peaks.bed//unioned_cleaned_peaks.bed_shuffle68.motif_presence.bed
	 1
	 /projects/ps-yeolab3/ekofman/Hugo/Full_RBFOX2_and_SLBP/9.4_shuffled_union_peaks/unioned_cleaned_peaks.bed/ unioned_cleaned_peaks.bed_shuffle42
		 /projects/ps-yeolab3/ekofman/Hugo/Full_RBFOX2_and_SLBP/9.4_shuffled_union_peaks/unioned_cleaned_peaks.bed//unioned_cleaned_peaks.bed_shuffle42.motif_presence.bed
	...Reading...
	Outputting file /projects/ps-yeolab3/ekofman/Hugo/Full_RBFOX2_and_SLBP/9.4_shuffled_union_peaks/unioned_cleaned_peaks.bed//unioned_cleaned_peaks.bed_shuffle42.motif_p

In [28]:
all_fractions

,unioned_cleaned_peaks.bed_shuffle68,unioned_cleaned_peaks.bed_shuffle42,unioned_cleaned_peaks.bed_shuffle8,unioned_cleaned_peaks.bed_shuffle59,unioned_cleaned_peaks.bed_shuffle72,unioned_cleaned_peaks.bed_shuffle64,unioned_cleaned_peaks.bed_shuffle97,unioned_cleaned_peaks.bed_shuffle23,unioned_cleaned_peaks.bed_shuffle1,unioned_cleaned_peaks.bed_shuffle77,...,unioned_cleaned_peaks.bed_shuffle38,unioned_cleaned_peaks.bed_shuffle16,unioned_cleaned_peaks.bed_shuffle95,unioned_cleaned_peaks.bed_shuffle76,unioned_cleaned_peaks.bed_shuffle27,unioned_cleaned_peaks.bed_shuffle37,unioned_cleaned_peaks.bed_shuffle66,unioned_cleaned_peaks.bed_shuffle32,unioned_cleaned_peaks.bed_shuffle71,unioned_cleaned_peaks.bed_shuffle74
TGCATG,0.043824,0.044818,0.041836,0.041113,0.044547,0.042378,0.044999,0.046535,0.04021,0.045179,...,0.04283,0.04283,0.041655,0.041926,0.042107,0.044095,0.042378,0.043463,0.04283,0.040661


In [29]:
pybedtools.cleanup()